In [1]:
'''
!pip install datasets
!pip install transformers[torch] accelerate -U
pip install pyttsx3
!apt-get update
!apt-get install -y espeak libespeak1
'''

'\n!pip install datasets\n!pip install transformers[torch] accelerate -U\npip install pyttsx3\n!apt-get update\n!apt-get install -y espeak libespeak1\n'

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Загрузка токенизатора и модели DistilRuBERT
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny2")

# Создание пайплайна для анализа текста
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Пример текста на русском языке
text = "Я очень счастлив сегодня!"

# Анализ текста
result = nlp(text)
print(result)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_0', 'score': 0.5212835073471069}]


In [3]:
'''
import pandas as pd
import requests
import zipfile
import io
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import Dataset

# Скачиваем и распаковываем датасет RuSentiment
url = "https://github.com/strawberrypie/rusentiment/archive/refs/heads/master.zip"
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

# Чтение данных
df = pd.read_csv("rusentiment-master/Dataset/rusentiment_preselected_posts.csv")

# Преобразование меток тональности в числовые значения
label_map = {"neutral": 0, "positive": 1, "negative": 2}
df['label'] = df['label'].map(label_map)

# Удаление строк с пустыми и недопустимыми значениями
df = df.dropna(subset=['text', 'label'])
df = df[df['label'].notna()]
df = df[df['label'] != 'inf']
df = df[df['label'] != '-inf']

# Преобразование меток в целые числа
df['label'] = df['label'].astype(int)

# Удаление строк с пустыми значениями
df = df.dropna(subset=['text', 'label'])

# Конвертация данных в формат, подходящий для обучения
train_dataset = Dataset.from_pandas(df[['text', 'label']])
'''

'\nimport pandas as pd\nimport requests\nimport zipfile\nimport io\nimport numpy as np\nfrom sklearn.metrics import accuracy_score, precision_recall_fscore_support\nfrom datasets import Dataset\n\n# Скачиваем и распаковываем датасет RuSentiment\nurl = "https://github.com/strawberrypie/rusentiment/archive/refs/heads/master.zip"\nr = requests.get(url)\nz = zipfile.ZipFile(io.BytesIO(r.content))\nz.extractall()\n\n# Чтение данных\ndf = pd.read_csv("rusentiment-master/Dataset/rusentiment_preselected_posts.csv")\n\n# Преобразование меток тональности в числовые значения\nlabel_map = {"neutral": 0, "positive": 1, "negative": 2}\ndf[\'label\'] = df[\'label\'].map(label_map)\n\n# Удаление строк с пустыми и недопустимыми значениями\ndf = df.dropna(subset=[\'text\', \'label\'])\ndf = df[df[\'label\'].notna()]\ndf = df[df[\'label\'] != \'inf\']\ndf = df[df[\'label\'] != \'-inf\']\n\n# Преобразование меток в целые числа\ndf[\'label\'] = df[\'label\'].astype(int)\n\n# Удаление строк с пустыми значен

In [4]:
'''
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

# Загрузка токенизатора и модели DistilRuBERT
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny2", num_labels=3)

# Токенизация данных
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = train_dataset.map(tokenize_function, batched=True)
print(tokenized_datasets)

# Проверка формата токенизированных данных
print(tokenized_datasets.features)
def format_labels(examples):
    examples['label'] = [int(label) for label in examples['label']]
    return examples

tokenized_datasets = tokenized_datasets.map(format_labels, batched=True)

# Настройки обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,   # Уменьшенный размер батча
    gradient_accumulation_steps=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
    acc = accuracy_score(p.label_ids, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Создание тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    compute_metrics=compute_metrics
)

# Обучение модели
trainer.train()
'''

'\nfrom transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments\nimport torch\n\n# Загрузка токенизатора и модели DistilRuBERT\ntokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")\nmodel = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny2", num_labels=3)\n\n# Токенизация данных\ndef tokenize_function(examples):\n    return tokenizer(examples["text"], padding="max_length", truncation=True)\n\ntokenized_datasets = train_dataset.map(tokenize_function, batched=True)\nprint(tokenized_datasets)\n\n# Проверка формата токенизированных данных\nprint(tokenized_datasets.features)\ndef format_labels(examples):\n    examples[\'label\'] = [int(label) for label in examples[\'label\']]\n    return examples\n\ntokenized_datasets = tokenized_datasets.map(format_labels, batched=True)\n\n# Настройки обучения\ntraining_args = TrainingArguments(\n    output_dir=\'./results\',\n    num_train_epochs=3,\n    per_device_tra

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
output_dir = './bert_fine_tuned_model'

'''
# Сохранение модели
model.save_pretrained(output_dir)

# Сохранение токенизатора
tokenizer.save_pretrained(output_dir)

print(f"Model and tokenizer saved to {output_dir}")
'''
# Загрузка сохраненной модели и токенизатора
tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModelForSequenceClassification.from_pretrained(output_dir)

In [6]:
'''
import shutil
from google.colab import files

# Путь к сохраненной модели и токенизатору
output_dir = './fine_tuned_model'

# Создание архива
shutil.make_archive('fine_tuned_model', 'zip', output_dir)

# Загрузка архива на локальный компьютер
files.download('fine_tuned_model.zip')
'''

"\nimport shutil\nfrom google.colab import files\n\n# Путь к сохраненной модели и токенизатору\noutput_dir = './fine_tuned_model'\n\n# Создание архива\nshutil.make_archive('fine_tuned_model', 'zip', output_dir)\n\n# Загрузка архива на локальный компьютер\nfiles.download('fine_tuned_model.zip')\n"

In [7]:
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Словарь для интерпретации меток
label_map = {0: "neutral", 1: "positive", 2: "negative"}

# Пример текста на русском языке
text = "Я очень счастлив сегодня!"

# Анализ текста
result = nlp(text)

# Интерпретация результата
def Interpritate_result(result):
  for r in result:
      r['label'] = label_map[int(r['label'].split('_')[-1])]  # Преобразование метки в читаемый формат

Interpritate_result(result)
print(result)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'positive', 'score': 0.9574131369590759}]


In [8]:
result = nlp('Слушай, это было круто.')
Interpritate_result(result)
print(result)

[{'label': 'positive', 'score': 0.8959010243415833}]


In [9]:
import pyttsx3
from random import *

def analyze_emotion(text):
    result = nlp(text)
    for r in result:
        numerical_label = int(r['label'].split('_')[-1])
        r['label'] = label_map[numerical_label]
    return result[0]['label'], result[0]['score']

def generate_audio(text, emotion):
    engine = pyttsx3.init()

    voices = engine.getProperty('voices')

    # Задать голос по умолчанию
    engine.setProperty('voice', 'ru')

    # Попробовать установить предпочтительный голос
    for voice in voices:
        if voice.name == 'Vsevolod':
            engine.setProperty('voice', voice.id)

    # Настройка параметров голоса в зависимости от эмоции
    if emotion == "positive":
        engine.setProperty('rate', 150)  # Скорость речи
        engine.setProperty('volume', 1.0)  # Громкость речи
    elif emotion == "negative":
        engine.setProperty('rate', 100)  # Скорость речи
        engine.setProperty('volume', 0.7)  # Громкость речи
    else:  # neutral
        engine.setProperty('rate', 120)  # Скорость речи
        engine.setProperty('volume', 0.9)  # Громкость речи
    
    output_path = f"output_{emotion}_{randint(0,10000)}.mp3"
    engine.save_to_file(text, output_path)
    engine.runAndWait()

    return output_path

def process_text_and_generate_audio(text):
    # Анализ текста на эмоции
    emotion, score = analyze_emotion(text)
    print(f"Emotion: {emotion}, Score: {score}")

    # Генерация аудиофайла на основе эмоции
    audio_file = generate_audio(text, emotion)
    print(f"Audio file generated: {audio_file}")

In [10]:
import re

def split_text_by_mood(text):
    # Пример простого деления по предложениям
    sentences = re.split(r'(?<=[.!?]) +', text)
    return sentences

In [11]:
def analyze_parts(parts):
    emotions = []
    for part in parts:
        emotion, score = analyze_emotion(part)
        emotions.append((part, emotion))
    return emotions

In [12]:
import os

def generate_audio_parts(parts_with_emotions):
    audio_files = []
    for i, (text, emotion) in enumerate(parts_with_emotions):
        audio_file = generate_audio(text, emotion)
        if os.path.exists(audio_file):
            print(f"Audio file generated: {audio_file}")
        else:
            print(f"Error: Audio file not generated: {audio_file}")
        audio_files.append(audio_file)
    return audio_files

In [17]:
import subprocess

def run_sadtalker(src_image, audio_file, output_file):
    command = [
        'python', 'inference.py',
        '--driven_audio', audio_file,
        '--source_image', src_image,
        '--result_dir', output_file
    ]
    subprocess.run(command)

def generate_video_parts(audio_files, parts_with_emotions):
    video_files = []
    src_image = "man.png"  # Изображение вашего аватара
    %cd ./../SadTalker
    for i, (audio_file, (text, emotion)) in enumerate(zip(audio_files, parts_with_emotions)):
        output_file = f"output_video_part_{i}"
        run_sadtalker(src_image, audio_file, f'./../DLS_project/result/{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}/{output_file}')
        output_video_path = os.path.join(output_file, 'result.mp4')
        if os.path.exists(output_video_path):
            print(f"Video file generated: {output_video_path}")
        else:
            print(f"Error: Video file not generated: {output_video_path}")
        video_files.append(output_video_path)
    %cd ./../DLS_project
    return video_files

In [18]:
from moviepy.editor import concatenate_videoclips, VideoFileClip

def combine_videos(video_files):
    clips = [VideoFileClip(video) for video in video_files]
    final_clip = concatenate_videoclips(clips)
    final_clip.write_videofile("final_output_video.mp4", codec="libx264")
    if os.path.exists("final_output_video.mp4"):
        print("Final video generated successfully: final_output_video.mp4")
    else:
        print("Error: Final video not generated")

In [19]:
def Make_avatar(text):
    parts = split_text_by_mood(text)
    print(parts)
    
    parts_with_emotions = analyze_parts(parts)
    print(parts_with_emotions)
    
    audio_files = generate_audio_parts(parts_with_emotions)
    print(audio_files)
    
    video_files = generate_video_parts(audio_files, parts_with_emotions)
    print(video_files)
    
    combine_videos(video_files)

In [20]:
# Пример использования
text = "Я очень счастлив сегодня! Но вчера мне было грустно. Завтра будет новый день, и я надеюсь, что он будет лучше."

Make_avatar(text)

['Я очень счастлив сегодня!', 'Но вчера мне было грустно.', 'Завтра будет новый день, и я надеюсь, что он будет лучше.']
[('Я очень счастлив сегодня!', 'positive'), ('Но вчера мне было грустно.', 'negative'), ('Завтра будет новый день, и я надеюсь, что он будет лучше.', 'positive')]
Audio file generated: output_positive_5843.mp3
Audio file generated: output_negative_1933.mp3
Audio file generated: output_positive_7145.mp3
['output_positive_5843.mp3', 'output_negative_1933.mp3', 'output_positive_7145.mp3']
D:\SadTalker
Error: Video file not generated: output_video_part_0\result.mp4
Error: Video file not generated: output_video_part_1\result.mp4
Error: Video file not generated: output_video_part_2\result.mp4
D:\DLS_project
['output_video_part_0\\result.mp4', 'output_video_part_1\\result.mp4', 'output_video_part_2\\result.mp4']


OSError: MoviePy error: the file output_video_part_0\result.mp4 could not be found!
Please check that you entered the correct path.